In [36]:
import numpy as np
import scipy.special
import scipy.optimize
import sklearn.datasets
class LinearLogisticRegression:

    def __init__(self, lbd, prior_weighted=False, prior=0.5):
        self.lbd = lbd
        self.prior_weighted = prior_weighted
        self.prior = prior


    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):
        w, b = v[0:-1], v[-1]
        z = 2 * self.Ltrain - 1
        exp = (np.dot(w.T, self.Dtrain) + b)
        reg = 0.5 * self.lbd * np.linalg.norm(w) ** 2
        avg_risk = (np.logaddexp(0, -exp * z)).mean()
        return reg + avg_risk

    def __logreg_obj_prior_weighted(self, v):
        w, b = v[0:-1], v[-1]
        z = 2 * self.Ltrain - 1
        reg = 0.5 * self.lbd * np.linalg.norm(w) ** 2
        exp = (np.dot(w.T, self.Dtrain) + b)
        avg_risk_0 = np.logaddexp(0, -exp[self.Ltrain == 0] * z[self.Ltrain == 0]).mean()*(1-self.prior)
        avg_risk_1 = np.logaddexp(0, -exp[self.Ltrain == 1] * z[self.Ltrain == 1]).mean() * self.prior
        return reg + avg_risk_0 + avg_risk_1
    def train(self, Dtrain, Ltrain):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        obj_function = self.__logreg_obj if self.prior_weighted is False else self.__logreg_obj_prior_weighted
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=obj_function,
                                                    x0=np.zeros(self.Dtrain.shape[0] + 1),
                                                    approx_grad=True,
                                                    iprint=0)

        """                                            
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        """
        return self

    def predict(self, Dtest, labels=True):
        w, b = self.x[0:-1], self.x[-1]
        S = np.zeros((Dtest.shape[1]))
        for i in range(Dtest.shape[1]):
            xi = Dtest[:, i:i + 1]
            s = np.dot(w.T, xi) + b
            S[i] = s
        if labels:
            LP = S > 0
            return LP
        else:
            return S

class QuadraticLogisticRegression:
    def __init__(self):
        pass

    def __compute_zi(self, ci):
        return 2 * ci - 1

    def __logreg_obj(self, v):  # still works if DTR is one sample only? yes but it must be of shape (4,1)
        w, b = v[0:-1], v[-1]
        J = self.lbd / 2 * (np.linalg.norm(w) ** 2)
        summary = 0
        for i in range(self.N):
            xi = self.Dtrain[:, i:i + 1]
            ci = self.Ltrain[i]
            zi = self.__compute_zi(ci)
            summary += np.logaddexp(0, -zi * (np.dot(w.T, xi) + b))
        J += (1 / self.N) * summary
        return J

    def train(self, Dtrain, Ltrain, lbd, maxiter):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.lbd = lbd
        self.maxiter = maxiter
        self.F = Dtrain.shape[0]  # dimensionality of features space
        self.K = len(set(Ltrain))  # number of classes
        self.N = Dtrain.shape[1]
        DTR_ext = numpy.hstack([self.__expandFeatures(self.Dtrain[:, i]) for i in range(self.Dtrain.shape[1])])
        self.x, f, d = scipy.optimize.fmin_l_bfgs_b(func=self.__logreg_obj, 
                                                    x0=numpy.zeros(DTR_ext.shape[0] + 1), 
                                                    approx_grad = True, 
                                                    factr=1.0)
        print('Point of minimum: %s' % (self.x))
        print('Value of the minimum: %s' % (f))
        print('Number of iterations: %s' % (d['funcalls']))
        return self    
    
    def __expandFeatures(x):
        x = utils.mcol(x)
        expX = utils.mcol(numpy.dot(x, x.T))
        return numpy.vstack([expX, x])

    def predict(self, Dtest):
        DTE_ext = numpy.hstack([self.__expandFeatures(Dtest[:, i]) for i in range(Dtest.shape[1])])
        w, b = utils.mcol(self.x[0:-1]), self.x[-1]
        scores = numpy.dot(w.T, DTE_ext) + b
        return scores[0, :]

In [74]:
def load_iris_binary():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    D = D[:, L != 0] # remove setosa from D
    L = L[L!=0] # remove setosa from L
    L[L==2] = 0 # We assign label 0 to virginica (was label 2)
    return D, L
def load_iris():
    D, L = sklearn.datasets.load_iris()['data'].T, sklearn.datasets.load_iris()['target']
    return D, L
def split_db_2to1(D, L, seed=0):
    nTrain = int(D.shape[1]*2.0/3.0) # 2/3 of the dataset D are used for training, 1/3 for validation
    np.random.seed(seed)
    idx = np.random.permutation(D.shape[1]) # take a random array of 150 elements, each element is 0<x<=149 (np.arange(150))
    idxTrain = idx[0:nTrain] # first 100 are indices of training samples
    idxTest = idx[nTrain:] # remaining 50 are indices of validation samples
    DTR = D[:, idxTrain] # D for training
    DTE = D[:, idxTest] # D for validation
    LTR = L[idxTrain] # L for training
    LTE = L[idxTest] # L for validation
    return (DTR, LTR), (DTE, LTE)

DT, LT = load_iris()
(DTR, LTR), (DTE, LTE) = split_db_2to1(DT, LT)

In [75]:
class SVM:
    def __init__(self, hparams, kernel=None):
        self.kernelType = kernel
        self.C = hparams['C']
        self.K = hparams['K']
        self.eps = hparams.get('eps')
        self.gamma = hparams.get('gamma')
        self.c = hparams.get('c')
        self.d = hparams.get('d')

    def __LDc_obj(self, alpha):
        #grad = np.dot(self.H, alpha) - np.ones(self.H.shape[1])
        t = 0.5 * np.dot(np.dot(alpha.reshape(1, -1), self.H), alpha) - alpha.sum(), np.dot(self.H, alpha) - 1
        return t

    def __polynomial_kernel(self, X1, X2):
        ker = (np.dot(X1.T, X2) + self.c) ** self.d + self.K ** 2
        return ker

    def __RBF_kernel(self, X1, X2):
        x = np.repeat(X1, X2.shape[1], axis=1)
        y = np.tile(X2, X1.shape[1])
        ker = np.exp(
            -self.gamma * np.linalg.norm(x - y, axis=0).reshape(X1.shape[1], X2.shape[1]) ** 2) + self.K ** 2
        return ker

    def train(self, Dtrain, Ltrain):
        self.Dtrain = Dtrain
        self.Ltrain = Ltrain
        self.N = Dtrain.shape[1]
        self.Ltrain_z = self.Ltrain * 2 - 1
        self.Ltrain_z_matrix = self.Ltrain_z.reshape(-1, 1) * self.Ltrain_z.reshape(1, -1)
        self.bounds = np.array([(0, self.C)] * Ltrain.shape[0])

        if self.kernelType is not None:
            if self.kernelType == 'Polynomial':
                ker = self.__polynomial_kernel(self.Dtrain, self.Dtrain)
            elif self.kernelType == 'RBF':
                ker = self.__RBF_kernel(self.Dtrain, self.Dtrain)
            else:
                return
            self.H = self.Ltrain_z_matrix * ker
        else:
            # Compute expanded D matrix
            self.expandedD = np.vstack((Dtrain, self.K * np.ones(self.N)))
            # Compute H matrix
            G = np.dot(self.expandedD.T, self.expandedD)
            self.H = G * self.Ltrain_z_matrix

        self.alpha, self.primal, _ = scipy.optimize.fmin_l_bfgs_b(func=self.__LDc_obj,
                                                                    bounds=self.bounds,
                                                                    x0=np.zeros(Dtrain.shape[1]),
                                                                    factr=1.0,
                                                                     maxfun=5000,
                                                                     maxiter=5000)

        return self

    def predict(self, Dtest, labels=False):
        if self.kernelType is not None:
            if self.kernelType == 'Polynomial':
                self.S = np.sum(np.dot((self.alpha * self.Ltrain_z).reshape(1, -1), self.__polynomial_kernel(self.Dtrain, Dtest)), axis=0)
            elif self.kernelType == 'RBF':
                self.S = np.sum(np.dot((self.alpha * self.Ltrain_z).reshape(1, -1), self.__RBF_kernel(self.Dtrain, Dtest)), axis=0)
            else:
                return
        else:
            wc = np.sum(self.alpha * self.Ltrain_z * self.expandedD, axis=1)
            self.w, self.b = wc[:-1], wc[-1::]
            self.S = np.dot(self.w.T, Dtest) + self.b * self.K

        if labels is True:
            predicted_labels = np.where(self.S > 0, 1, 0)
            return predicted_labels
        else:
            return self.S

In [76]:
def read_data_training(path):
    DT = np.zeros(shape=(12, 6000), dtype='float32')  # DT: Data Training
    LT = np.zeros(6000, dtype='int32')  # LT: Labels Training
    with open(path, 'r') as file:
        i = 0
        for line in file:
            features_list = line.split(',')
            features = np.array(features_list[0:12], dtype='float32').reshape(-1, 1)
            label = int(features_list[-1])
            DT[:, i:i + 1] = features
            LT[i] = label
            i += 1
    return DT, LT

def read_data_evaluation(path):
    DE = np.zeros(shape=(12, 4000), dtype='float32')  # DT: Data Training
    LE = np.zeros(4000, dtype='int32')  # LT: Labels Training
    with open(path, 'r') as file:
        i = 0
        for line in file:
            features_list = line.split(',')
            features = np.array(features_list[0:12], dtype='float32').reshape(-1, 1)
            label = int(features_list[-1])
            DE[:, i:i + 1] = features
            LE[i] = label
            i += 1
    return DE, LE

In [77]:
#DT, LT = read_data_training('Train.txt')
#DE, LE = read_data_evaluation('Test.txt')

In [78]:
import numpy as np
import scipy.special


class GMM:
    def __init__(self):
        pass

    def logpdf_GAU_ND(self, X, mu, C):
        invC = np.linalg.inv(C)
        _, log_abs_detC = np.linalg.slogdet(C)
        M = X.shape[0]
        return - M/2 * np.log(2 * np.pi) - 0.5 * log_abs_detC - 0.5 * ((X - mu) * np.dot(invC, X - mu)).sum(0)

    def logpdf_GMM(self, X, gmm):
        S = np.zeros((len(gmm), X.shape[1]))

        for g in range(len(gmm)):
            (w, mu, C) = gmm[g]
            S[g, :] = self.logpdf_GAU_ND(X, mu, C) + np.log(w)

        logdens = scipy.special.logsumexp(S, axis=0)
        return S, logdens

    def GMM_algorithm_EM(self, X, gmm, psi=0.01, cov='Full'):
        thNew = None
        thOld = None
        N = X.shape[1]
        D = X.shape[0]

        while thOld == None or thNew - thOld > 1e-6:
            thOld = thNew
            logSj, logSjMarg = self.logpdf_GMM(X, gmm)
            thNew = np.sum(logSjMarg) / N

            P = np.exp(logSj - logSjMarg)

            if cov == 'Diag':
                newGmm = []
                for i in range(len(gmm)):
                    gamma = P[i, :]
                    Z = gamma.sum()
                    F = (gamma.reshape(1, -1) * X).sum(1)
                    S = np.dot(X, (gamma.reshape(1, -1) * X).T)
                    w = Z / N
                    mu = (F / Z).reshape(-1, 1)
                    sigma = S / Z - np.dot(mu, mu.T)
                    sigma *= np.eye(sigma.shape[0])
                    U, s, _ = np.linalg.svd(sigma)
                    s[s < psi] = psi
                    sigma = np.dot(U, s.reshape(-1, 1) * U.T)
                    newGmm.append((w, mu, sigma))
                gmm = newGmm

            elif cov == 'Tied':
                newGmm = []
                sigmaTied = np.zeros((D, D))
                for i in range(len(gmm)):
                    gamma = P[i, :]
                    Z = gamma.sum()
                    F = (gamma.reshape(1, -1) * X).sum(1)
                    S = np.dot(X, (gamma.reshape(1, -1) * X).T)
                    w = Z / N
                    mu = (F / Z).reshape(-1, 1)
                    sigma = S / Z - np.dot(mu, mu.T)
                    sigmaTied += Z * sigma
                    newGmm.append((w, mu))
                gmm = newGmm
                sigmaTied /= N
                U, s, _ = np.linalg.svd(sigmaTied)
                s[s < psi] = psi
                sigmaTied = np.dot(U, s.reshape(-1, 1) * U.T)

                newGmm = []
                for i in range(len(gmm)):
                    (w, mu) = gmm[i]
                    newGmm.append((w, mu, sigmaTied))

                gmm = newGmm

            elif cov == 'TiedDiag':
                newGmm = []
                sigmaTied = np.zeros((D, D))
                for i in range(len(gmm)):
                    gamma = P[i, :]
                    Z = gamma.sum()
                    F = (gamma.reshape(1, -1) * X).sum(1)
                    S = np.dot(X, (gamma.reshape(1, -1) * X).T)
                    w = Z / N
                    mu = (F / Z).reshape(-1, 1)
                    sigma = S / Z - np.dot(mu, mu.T)
                    sigmaTied += Z * sigma
                    newGmm.append((w, mu))
                gmm = newGmm
                sigmaTied /= N
                sigmaTied *= np.eye(sigma.shape[0])
                U, s, _ = np.linalg.svd(sigmaTied)
                s[s < psi] = psi
                sigmaTied = np.dot(U, s.reshape(-1, 1) * U.T)

                newGmm = []
                for i in range(len(gmm)):
                    (w, mu) = gmm[i]
                    newGmm.append((w, mu, sigmaTied))

                gmm = newGmm

            else:
                newGmm = []
                for i in range(len(gmm)):
                    gamma = P[i, :]
                    Z = gamma.sum()
                    F = (gamma.reshape(1, -1) * X).sum(1)
                    S = np.dot(X, (gamma.reshape(1, -1) * X).T)
                    w = Z / N
                    mu = (F / Z).reshape(-1, 1)
                    sigma = S / Z - np.dot(mu, mu.T)
                    U, s, _ = np.linalg.svd(sigma)
                    s[s < psi] = psi
                    sigma = np.dot(U, s.reshape(-1, 1) * U.T)
                    newGmm.append((w, mu, sigma))
                gmm = newGmm
        return gmm

    def GMM_algorithm_LBG(self, X, alpha, nComponents, psi=0.01, covType='Full'):
        mean = X.mean(axis=1).reshape(-1, 1)
        cov = 1 / X.shape[1] * np.dot(X - mean, (X - mean).T)
        gmm = [(1, mean, cov)]

        while len(gmm) <= nComponents:
            gmm = self.GMM_algorithm_EM(X, gmm, psi, covType)

            if len(gmm) == nComponents:
                break

            newGmm = []
            for i in range(len(gmm)):
                (w, mu, sigma) = gmm[i]
                U, s, Vh = np.linalg.svd(sigma)
                d = U[:, 0:1] * s[0] ** 0.5 * alpha
                newGmm.append((w / 2, mu + d, sigma))
                newGmm.append((w / 2, mu - d, sigma))
            gmm = newGmm

        return gmm

    def trainGMM(self, DTR, LTR, DTE, alpha, nComponents, psi=0.01, covType='Full'):
        DTR_0 = DTR[:, LTR == 0]
        gmm_c0 = self.GMM_algorithm_LBG(DTR_0, alpha, nComponents, psi, covType)
        _, llr_0 = self.logpdf_GMM(DTE, gmm_c0)

        DTR_1 = DTR[:, LTR == 1]
        gmm_c1 = self.GMM_algorithm_LBG(DTR_1, alpha, nComponents, psi, covType)
        _, llr_1 = self.logpdf_GMM(DTE, gmm_c1)

        return llr_1 - llr_0

In [96]:
g = GMM()
alpha=0.1
nComponents = 4
psi = 0.01
covType='Diag'
gmm_c0 = g.GMM_algorithm_LBG(DTR[:, LTR==0], alpha, nComponents, psi, covType)
_, llr_0 = g.logpdf_GMM(DTE, gmm_c0)
gmm_c1 = g.GMM_algorithm_LBG(DTR[:, LTR==1], alpha, nComponents, psi, covType)
_, llr_1 = g.logpdf_GMM(DTE, gmm_c1)
gmm_c2 = g.GMM_algorithm_LBG(DTR[:, LTR==2], alpha, nComponents, psi, covType)
_, llr_2 = g.logpdf_GMM(DTE, gmm_c2)

In [97]:
S = np.vstack([llr_0.reshape(1, -1), llr_1.reshape(1, -1), llr_2.reshape(1, -1)])
predicted = np.argmax(S, axis=0)

In [98]:
print('Error rate: ', (1-sum(predicted == LTE)/len(LTE))*100,'%%')

Error rate:  6.000000000000005 %%
